In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env

binance_api_key = os.getenv('BINANCE_API_KEY')
binance_secret_key = os.getenv('BINANCE_SECRET_KEY')

In [ ]:
# 2. Connect to Binance (with python-binance)

In [2]:
from binance.client import Client

client = Client(binance_api_key, binance_secret_key)
# Example: get latest prices
prices = client.get_all_tickers()
print(prices[:3])

[{'symbol': 'ETHBTC', 'price': '0.02456000'}, {'symbol': 'LTCBTC', 'price': '0.00082500'}, {'symbol': 'BNBBTC', 'price': '0.00606500'}]


In [ ]:
# 3. Send Data to Kafka

In [3]:
from kafka import KafkaProducer
import json

producer = KafkaProducer(
    bootstrap_servers='kafka:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Send prices to Kafka topic "binance_prices"
producer.send('binance_prices', prices)
producer.flush()


In [6]:
# 4. Test the End-to-End Pipeline
from dotenv import load_dotenv
import os
from binance.client import Client
from kafka import KafkaProducer
import json

# Load .env
load_dotenv()

# Connect to Binance
api_key = os.getenv('BINANCE_API_KEY')
secret_key = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, secret_key)

# Fetch data
prices = client.get_all_tickers()

# Set up Kafka producer
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',  # or your Kafka host
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Send data
producer.send('binance_prices', prices)
producer.flush()
print("Data sent to Kafka topic 'binance_prices'")


Data sent to Kafka topic 'binance_prices'


In [ ]:
from dotenv import load_dotenv
import os
from binance.client import Client
from kafka import KafkaProducer
import json

# Load .env
load_dotenv()

# Connect to Binance
api_key = os.getenv('BINANCE_API_KEY')
secret_key = os.getenv('BINANCE_SECRET_KEY')
client = Client(api_key, secret_key)

# Fetch all prices
all_prices = client.get_all_tickers()

# Filter for BTC/USDT and ETH/USDT only
symbols_to_send = ['BTCUSDT', 'ETHUSDT']
filtered_prices = [p for p in all_prices if p['symbol'] in symbols_to_send]

# Set up Kafka producer
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',  # or your Kafka host
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Send filtered data
producer.send('binance_prices', filtered_prices)
producer.flush()
print("Data sent to Kafka topic 'binance_prices':", filtered_prices)


In [7]:
# 5. Visualize or Consume Data from Kafka
from kafka import KafkaConsumer
import json

consumer = KafkaConsumer(
    'binance_prices',
    bootstrap_servers='kafka:9092',
    auto_offset_reset='earliest',
    group_id='binance-test',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

for message in consumer:
    print(message.value)
    break  # Remove or adjust for continuous reading


[{'symbol': 'ETHBTC', 'price': '0.02456000'}, {'symbol': 'LTCBTC', 'price': '0.00082500'}, {'symbol': 'BNBBTC', 'price': '0.00606500'}, {'symbol': 'NEOBTC', 'price': '0.00005720'}, {'symbol': 'QTUMETH', 'price': '0.00081700'}, {'symbol': 'EOSETH', 'price': '0.00030670'}, {'symbol': 'SNTETH', 'price': '0.00001700'}, {'symbol': 'BNTETH', 'price': '0.00020260'}, {'symbol': 'BCCBTC', 'price': '0.00000000'}, {'symbol': 'GASBTC', 'price': '0.00002800'}, {'symbol': 'BNBETH', 'price': '0.24690000'}, {'symbol': 'BTCUSDT', 'price': '109409.60000000'}, {'symbol': 'ETHUSDT', 'price': '2686.88000000'}, {'symbol': 'HSRBTC', 'price': '0.00000000'}, {'symbol': 'OAXETH', 'price': '0.00000000'}, {'symbol': 'DNTETH', 'price': '0.00000000'}, {'symbol': 'MCOETH', 'price': '0.00000000'}, {'symbol': 'ICNETH', 'price': '0.00000000'}, {'symbol': 'MCOBTC', 'price': '0.00000000'}, {'symbol': 'WTCBTC', 'price': '0.00000024'}, {'symbol': 'WTCETH', 'price': '0.00000000'}, {'symbol': 'LRCBTC', 'price': '0.00000081'}